In [17]:
#Import all the package 

import urllib3
import os
import requests
import re
from bs4 import BeautifulSoup
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [18]:
#create soip and check status of url
url_imdb="https://www.imdb.com/chart/top?ref_=nv_mv_250"
req = requests.get(url_imdb)

print(req.status_code)
soup = BeautifulSoup(req.text, 'lxml')
print(soup.title.text)

200
IMDb Top 250 - IMDb


In [19]:
#Get all links
links=[]
for a in soup.find_all("a"):
    links.append(a.get("href"))
links=['https://www.imdb.com'+a.strip() for a in links if a is not None and a.startswith('/title/tt') ]

#Remove duplicate links
top_250M=[]
for c in links:
    if c not in top_250M:
        top_250M.append(c)
print(len(top_250M))
    


250


In [20]:
#Get the header, first create the list and in list get only text
column_list=['Rank','Movie_name' ,'URL' ,'Release_Year' ,'IMDB_Rating' ,
'Reviewer_count' ,'Censor_Board_Rating' ,'Movie_Length' ,'Genre_1' ,
'Genre_2' ,'Genre_3' ,'Genre_4' ,'Release_Date' ,'Story_Summary' ,
'Director' ,'Writer_1' ,'Writer_2' ,'Writer_3' ,'Star_1' ,
'Star_2' ,'Star_3' ,'Star_4' ,'Star_5' ,'Plot_Keywords' ,'Budget' ,
'Gross_USA' ,'Cum_Worldwide_Gross' ,'Production_Company' 
]
df = pd.DataFrame(columns=column_list)#,index=t) 
#df

In [23]:
#Get the Data 

for x in range(0,len(top_250M)):
    #----------------------------------load each page 
    URL =top_250M[x]
    req_m=requests.get(URL)
    #print(req_m.status_code)
    m_soup=BeautifulSoup(req_m.text,'lxml')
    
    
    #---------------------------------- get details of each movie
    #----- Movie_name, Movie_Length,Movie_type,Release_Year,IMDB_Rating,Reviewer_count,Story_Summary
    
    
    Movie_name=(m_soup.find("div",attrs={"class":"title_wrapper"})).get_text(strip=True).split("|")[0].split("(")[0]
    Movie_Length=(m_soup.find("div",attrs={"class":"title_wrapper"})).get_text(strip=True).split("|")[1]
    movie_Type=(m_soup.find("div",attrs={"class":"title_wrapper"})).get_text(strip=True).split("|")[2]
    Release_Year=(m_soup.find("div",attrs={"class":"title_wrapper"})).get_text(strip=True).split("|")[0].split("(")[1].split(')')[0]
    #year_released=((soup.find("div",{"class":"title_wrapper"}).get_text(strip=True).split('|')[0]).split('(')[1]).split(')')[0]
    IMDB_Rating=m_soup.find("span",attrs={"itemprop":"ratingValue"}).get_text()
    Reviewer_count=m_soup.find("span",attrs={"class":"small"}).get_text()
    Story_Summary=m_soup.find("div",attrs={"class":"summary_text"}).get_text().strip()
    
    # Getting Gener
    
    
    
    subtext= m_soup.find("div",{"class":"subtext"}).get_text(strip=True).split('|') #Censor_rating
    if len(subtext)<4:
        censor_rating='Not Rated'
        movie_len=subtext[0]
        genre_list=subtext[1].split(',')
        while len(genre_list)<4:         genre_list.append(" ")
        genre_1,genre_2,genre_3,genre_4=genre_list
        release_date=subtext[2]
    else:
        censor_rating=subtext[0]
        movie_len=subtext[1]
        genre_list=subtext[2].split(',')
        while len(genre_list)<4:         genre_list.append(" ")
        genre_1,genre_2,genre_3,genre_4=genre_list
        release_date=subtext[3]
        
    
    # Getting the list of Starts, writters and director
    Credit_Summary=m_soup.find_all("div",attrs={"class":"credit_summary_item"})
    b=[]
    for item in Credit_Summary:
        c=re.split(',|:|\|',item.get_text(strip=True))
        b.append(c)
    #print(b)
    
    #Getting popup Star,Writter and Directod in separate variable list
    stars=b.pop()
    writers=b.pop()
    directors=b.pop()
    
    #Removing the irrelevant text from each list-------------------------------------------------------------
   
    if "See full cast & crew»" in stars : stars.remove("See full cast & crew»")
    if '1 more credit»' in writers: writers.remove('1 more credit»') 
    if '1 more credit»' in directors: directors.remove('1 more credit»')
    stars=stars[1:]
    writers=writers[1:]
    directors=directors[1:]
    while len(stars)<5:         stars.append(" ")
    while len(writers)<3:         writers.append(" ")
        
        
    director=directors[0]
    star_1,star_2,star_3,star_4,star_5=stars
    writer_1,writer_2,writer_3=writers
    
    
    
      #Get PLot list ------------------------------------------------------------------------------------
    b=[]
    plot=m_soup.find_all("span",attrs={"class":"itemprop"})
    for item in plot:
        b.append(item.get_text(strip=True))
    plot_keywords='|'.join(b)
   
    #print(plot_keywords)
    
    
    # Get 'Budget','Gross_USA','Cum_Worldwide_Gross','Production_Company' -----------------------------------
    b=[]
    details=m_soup.find_all("div",attrs={"class":"txt-block"})
    #print(details)
    for item in details:
        b.append(item.get_text(strip=True).split(':'))
        
    #for i in range(0,len(b)):
        #print( b[i])
    d={'Budget':'', 'Opening Weekend USA':'','Gross USA':'','Cumulative Worldwide Gross':'','Production Co':'','Country':"",'Language':"",
      'Language':""}
    for i in b:
        if i[0] in d:
            d.update({i[0]:i[1]})
    #print(d)
    Budget=d['Budget']
    production_company=d['Production Co'].split('See more')[0]
    cum_world_gross=d['Cumulative Worldwide Gross'].split(' ')[0]
    gross_usa=d['Gross USA'].split(' ')[0]
    
    
    movie_dict={
        'Rank':x+1,
        'Movie_name' : Movie_name,
        'URL' : URL,
        'Release_Year' : Release_Year,
        'IMDB_Rating' : IMDB_Rating,
        'Reviewer_count' : Reviewer_count,
        'Censor_Board_Rating' : censor_rating,
        'Movie_Length' : Movie_Length,
        'Genre_1' : genre_1,
        'Genre_2' : genre_2,
        'Genre_3' : genre_3,
        'Genre_4' : genre_4,
        'Release_Date' : release_date,
        'Story_Summary' : Story_Summary,
        'Director' : director,
        'Writer_1' : writer_1,
        'Writer_2' : writer_2,
        'Writer_3' : writer_3,
        'Star_1' : star_1,
        'Star_2' : star_2,
        'Star_3' : star_3,
        'Star_4' : star_4,
        'Star_5' : star_5,
        'Plot_Keywords' : plot_keywords,
        'Budget' : Budget,
        'Gross_USA' : gross_usa,
        'Cum_Worldwide_Gross' : cum_world_gross,
        'Production_Company' : production_company
        }       
        
    #print(movie_dict)
    #print("*"*100)
    #print(b[13])
    df = df.append(pd.DataFrame.from_records([movie_dict],columns=movie_dict.keys() ) )
    df
    print(type(df))

    

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pand

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pand

In [24]:
df.to_excel("IMDB_Data_self.xlsx","Sheet",index=False)

In [16]:
df

,Rank,Movie_name,URL,Release_Year,IMDB_Rating,Reviewer_count,Censor_Board_Rating,Movie_Length,Genre_1,Genre_2,...,Star_1,Star_2,Star_3,Star_4,Star_5,Plot_Keywords,Budget,Gross_USA,Cum_Worldwide_Gross,Production_Company
0,1,The Shawshank Redemption,https://www.imdb.com/title/tt0111161/,1994,9.3,"2,290,646",A,2h 22min,Drama,,...,Tim Robbins,Morgan Freeman,Bob Gunton,,,wrongful imprisonment|based on the works of st...,"$25,000,000(estimated)","$28,699,976","$28,815,291",Castle Rock Entertainment
0,1,The Shawshank Redemption,https://www.imdb.com/title/tt0111161/,1994,9.3,"2,290,646",A,2h 22min,Drama,,...,Tim Robbins,Morgan Freeman,Bob Gunton,,,wrongful imprisonment|based on the works of st...,"$25,000,000(estimated)","$28,699,976","$28,815,291",Castle Rock Entertainment
0,2,The Godfather,https://www.imdb.com/title/tt0068646/,1972,9.2,"1,580,997",A,2h 55min,Crime,Drama,...,Marlon Brando,Al Pacino,James Caan,,,crime family|mafia|patriarch|organized crime|r...,"$6,000,000(estimated)","$134,966,411","$246,120,974","Paramount Pictures,Alfran Productions"
0,3,The Godfather: Part II,https://www.imdb.com/title/tt0071562/,1974,9.0,"1,104,576",A,3h 22min,Crime,Drama,...,Al Pacino,Robert De Niro,Robert Duvall,,,revenge|corrupt politician|bloody body of chil...,"$13,000,000(estimated)","$47,834,595","$48,035,783","Paramount Pictures,The Coppola Company,America..."
0,4,The Dark Knight,https://www.imdb.com/title/tt0468569/,2008,9.0,"2,255,777",UA,2h 32min,Action,Crime,...,Christian Bale,Heath Ledger,Aaron Eckhart,,,dc comics|joker|psychopath|clown|criminal mast...,"$185,000,000(estimated)","$535,234,033","$1,005,455,497","Warner Bros.,Legendary Entertainment,Syncopy"
0,5,12 Angry Men,https://www.imdb.com/title/tt0050083/,1957,8.9,"672,801",Not Rated,"Crime,Drama",Crime,Drama,...,Henry Fonda,Lee J. Cobb,Martin Balsam,,,jury|dialogue driven|courtroom|single set prod...,"$350,000(estimated)",,$576,Orion-Nova Productions
0,6,Schindler's List,https://www.imdb.com/title/tt0108052/,1993,8.9,"1,189,184",UA,3h 15min,Biography,Drama,...,Liam Neeson,Ralph Fiennes,Ben Kingsley,,,accountant|champagne|villa|womanizer|soap,"$22,000,000(estimated)","$96,898,818","$322,287,794","Universal Pictures,Amblin Entertainment"
0,7,The Lord of the Rings: The Return of the King,https://www.imdb.com/title/tt0167260/,2003,8.9,"1,611,246",U,3h 21min,Action,Adventure,...,Elijah Wood,Viggo Mortensen,Ian McKellen,,,orc|epic|battle|journey|hobbit,"$94,000,000(estimated)","$377,845,905","$1,142,270,989","New Line Cinema,WingNut Films,The Saul Zaentz ..."
0,8,Pulp Fiction,https://www.imdb.com/title/tt0110912/,1994,8.9,"1,788,917",A,2h 34min,Crime,Drama,...,John Travolta,Uma Thurman,Samuel L. Jackson,,,nonlinear timeline|overdose|drug overdose|bond...,"$8,000,000(estimated)","$107,928,762","$214,195,434","Miramax,A Band Apart,Jersey Films"
0,9,"Il buono, il brutto, il cattivo",https://www.imdb.com/title/tt0060196/,1966,8.8,"675,781",A,2h 41min,Western,,...,Clint Eastwood,Eli Wallach,Lee Van Cleef,,,spaghetti western|shootout|famous opening them...,"$1,200,000(estimated)","$25,100,000","$25,252,481","Produzioni Europee Associate (PEA),Arturo Gonz..."
